In [5]:
#

#1, 1, 2, 3, 5, 8, 13, 21, 34, ... , Fn+1 = Fn + Fn-1
#           2  1  1

#Fn+1 >= 2*Fn-1
#Fn+1 <= 2*Fn

#Fn+1 = 2 Fn [2^n] 
#Fn+1 = 2 Fn-1 [2^{n/2}]

#1, 2, 3, 4 ... [n]

def Fibonacci_num (n):
    if (n == 0 or n == 1):
        return 1
    
    return Fibonacci_num (n - 1) + Fibonacci_num (n - 2)

for i in range (38):
    print (i, Fibonacci_num (i))

0 1
1 1
2 2
3 3
4 5
5 8
6 13
7 21
8 34
9 55
10 89
11 144
12 233
13 377
14 610
15 987
16 1597
17 2584
18 4181
19 6765
20 10946
21 17711
22 28657
23 46368
24 75025
25 121393
26 196418
27 317811
28 514229
29 832040
30 1346269
31 2178309
32 3524578
33 5702887
34 9227465
35 14930352
36 24157817
37 39088169


In [19]:
import cv2
import numpy as np
import time
import math

WIND_X = 1200
WIND_Y = 1100

class Branch:
    def __init__(self, length_, angle_, color_, angle_range_,
                 length_decrement_factor_, max_depth_, branching_factor_):
        self.length = length_ #+ np.random.randint (int (length_ / 3)) - \
                             #int (length_ / 6)
        self.angle  = angle_ + float (np.random.randint (300) - 150) / 10000
        
        self.color  = color_
        
        if (max_depth_ < 1):
            self.color = (100, 230, 230)
        
        self.children = []
        
        if (max_depth_ > 0):
            angle_step = angle_range_ / (branching_factor_ - 1)
            
            for i in range (branching_factor_):
                self.add_child (length_ = int (self.length * length_decrement_factor_),
                                angle_  = - angle_range_ / 2 +
                                    self.angle + i * angle_step,
                                color_  = self.color,
                                angle_range_ = angle_range_,
                                length_decrement_factor_ = length_decrement_factor_,
                                max_depth_ = max_depth_ - 1,
                                branching_factor_ = branching_factor_)

    def add_child (self, length_, angle_, color_, angle_range_,
                   length_decrement_factor_, max_depth_, branching_factor_):
        new_branch = Branch (length_, angle_, color_, angle_range_,
                             length_decrement_factor_,
                             max_depth_, branching_factor_)
        
        self.children.append (new_branch)
    
    def draw (self, img, x, y, depth, tick):
        x1 = int (x + self.length * math.cos (self.angle))
        y1 = int (y + self.length * math.sin (self.angle))

        cv2.line (img, (x, y), (x1, y1), self.color, max(1, int(self.length / 65)))
        
        if (depth == 0):
            cv2.circle (img, (int (x1), int (y1)), 3, ((10 + tick * 7) % 255 + 10,
                        40, 10), -1)

        for child in self.children:
            child.draw (img, x1, y1, depth - 1, tick)

canvas_ = np.zeros ((WIND_Y, WIND_X, 3), np.uint8) * 75
canvas = canvas_.copy ()

tick = 0
shift = 4

cv2.namedWindow("render", cv2.WINDOW_AUTOSIZE)

length = 170
angle_range = 281
dec_fac = 62
max_depth = 3
branching = 4

def change_length (new_len):
    global length
    length = new_len

cv2.createTrackbar('length', 'render', length, 300, change_length)

def change_range (new_range):
    global angle_range
    angle_range = new_range

cv2.createTrackbar('range', 'render', angle_range, 314, change_range)

def change_dec (new_dec):
    global dec_fac
    dec_fac = new_dec

cv2.createTrackbar('decrement_factor', 'render', dec_fac, 150, change_dec)

def change_depth (new_depth):
    global max_depth
    max_depth = new_depth

cv2.createTrackbar('depth', 'render', max_depth, 7, change_depth)

def change_branching (new_branching):
    global branching
    branching = new_branching

cv2.createTrackbar('branching', 'render', branching, 7, change_branching)

while (True):
    tick += 1
    
    tree = Branch (length_ = length + 70 * math.sin (tick *  0.02),
                   angle_ = -1.57,
                   #color_ = (10, 20, (10 + tick * 7) % 255),
                   color_ = (10, (10 + 240) % 255, 50),
                   angle_range_ = angle_range / 100,
                   length_decrement_factor_ = dec_fac / 100,
                   max_depth_ = max_depth,
                   branching_factor_ = branching)
    
#     root = Branch (length_ = 140 + 30 * math.sin (tick *  0.05),
#                    angle_ = 1.57,
#                    #color_ = (10, 20, (10 + tick * 7) % 255),
#                    color_ = (20 % 255, 50, 90),
#                    angle_range_ = 2.81, length_decrement_factor_ = 0.62,
#                    max_depth_ = 3, branching_factor_ = 4)

    #if (tick > 0):
    #canvas = canvas_.copy ()

    tree.draw (canvas, 600, 1000, 2, tick)
    #root.draw (canvas, int (440), 700, 2, tick)
        
    shadow = canvas.astype ("float") * 0.95
    new_canvas = canvas_.copy ()

    new_canvas [:, shift :, :] = shadow [:, : -shift, :]

    canvas = new_canvas

    cv2.imshow ("render", canvas)

    time.sleep (0.05)
    
    key = cv2.waitKey (1) & 0xFF
    
    if (key == ord ('q')):
        break
    
    #break
    
cv2.destroyAllWindows ()

In [ ]:
import numpy as np
import cv2
import math
from tqdm import tqdm

WIND_X, WIND_Y = 900, 600

def Mandelbrot_fractal (x, y, xsz, img):
    sh = img.shape

    for i in tqdm (range (sh [1])):
        for j in range (sh [0]):
            #x0 = (i - sh [1] / 3 * 2) / 250
            #y0 = (j - sh [0] / 2) / 250
            x0 = (i - sh [1] / 7 * 12) / 1550
            y0 = (j - sh [0] / 7 * 6) / 1550

            x, y = 0.0, 0.0

            iteration, max_iteration = 0, 200

            while (x**2 + y**2 <= 2*2 and iteration < max_iteration):
                xtemp = x**2 - y**2 + x0
                y = 2 * x * y + y0
                x = xtemp
                iteration += 1

            if (iteration < 199):
                img [j, i, :] = ((240 + int (170 * math.log2 (iteration))) % 255,
                                 (100 + int (50 * math.log2 (iteration))) % 255,
                                 (50  + int (0)) % 255)

    return img

frame = Mandelbrot_fractal (1, 0, 2, np.zeros ((WIND_Y, WIND_X, 3), np.uint8))

cv2.imshow ("render", frame)

cv2.waitKey (0)


100%|██████████| 900/900 [00:24<00:00, 36.43it/s]
